1. Sword & Shield is about to release. New pokemons are coming. We want to help pokemon trainers to select pokemons in the new game.
2. draw a line plot/barplot to show the distribution of different types of pokemons. Guess what kind of pokemon will appear in the new game. So we can advice them what kind of pokemons they should catch when they go through the play.(this part is like kaggle's thing)
3. analysing the battle data. what is the critical stats of pokemon to win?
    3.1 mega pokemon have higher winnning rate
    3.2 eliminate mega pokemon, who has the highest winning rate?
    3.3 why there are some legendary pokemon have lower winning rate?
    3.4 how should trainer choose their pokemon during a battle?
        3.4.1 types
        3.4.2 stats
        3.4.3 damage
        3.4.4 defense
        3.4.5 speed
    
4. use simulation winning rate to prove the point.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt # data visualization
#import matplotlib.pyplot as plt
import seaborn as sns #data visualization
import random 
random.seed(888)

In [2]:
# Import the dataset
pokemon = pd.read_csv("pokemon/pokemon.csv")
# rename the column with a pound sign/hashtag as "number" its name
# The reason for this is when  we try and access this column later it will comment out the code
pokemon = pokemon.rename(index=str, columns={"#": "Number"})
combat = pd.read_csv("pokemon/combats.csv")
pokemon.head()

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [3]:
pokemon.isnull().sum()

Number          0
Name            1
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [4]:
pokemon[pokemon['Name'].isnull()]

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
62,63,NaN,Fighting,NaN,65,105,60,60,70,95,1,False


Check the Number of the pokemon, and get the missing value: Primeape

In [5]:
pokemon.loc[pokemon['Name'].isnull(),'Name']='Primeape'

Why there are missing value?  
Explain:

In [6]:
typelist = set(list(pokemon['Type 1']))

In [7]:
gen={}
for i in range(1,7):
    typep={}
    for j in typelist:
        typep[j]=sum((pokemon['Generation']==i) & ((pokemon['Type 1']==j) | (pokemon['Type 2']==j)))
    gen[i]=typep

In [8]:
type_gen=pd.DataFrame.from_dict(gen)
type_gen

,1,2,3,4,5,6
Bug,14,12,14,11,18,3
Dark,1,8,13,7,16,6
Dragon,4,2,15,8,12,9
Electric,9,9,5,12,12,3
Fairy,5,8,8,2,3,14
Fighting,9,4,9,10,17,4
Fire,14,11,9,6,16,8
Flying,23,19,14,16,21,8
Ghost,4,1,8,9,9,15
Grass,15,10,18,17,20,15


Combat part

In [9]:
# calculate the win % of each pokemon 
# add the calculation to the pokemon dataset 
total_Wins = combat.Winner.value_counts()
# get the number of wins for each pokemon
numberOfWins = combat.groupby('Winner').count()

#both methods produce the same results
countByFirst = combat.groupby('Second_pokemon').count()
countBySecond = combat.groupby('First_pokemon').count()

In [10]:
numberOfWins = numberOfWins.sort_index()
numberOfWins['Total Fights'] = countByFirst.Winner + countBySecond.Winner
numberOfWins['Win Percentage']= numberOfWins.First_pokemon/numberOfWins['Total Fights']

# merge the winning dataset and the original pokemon dataset
results2 = pd.merge(pokemon, numberOfWins, right_index = True, left_on='Number')
results3 = pd.merge(pokemon, numberOfWins, left_on='Number', right_index = True, how='left')
pokemon=results3

In [11]:
pokemon.head()

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,First_pokemon,Second_pokemon,Total Fights,Win Percentage
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,37.0,37.0,133.0,0.278195
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,46.0,46.0,121.0,0.380165
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,89.0,89.0,132.0,0.674242
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,70.0,70.0,125.0,0.560000
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,55.0,55.0,112.0,0.491071


In [12]:
pokemon['Defense_adj']=pokemon['Defense']*pokemon['HP']
pokemon['SpDef_adj']=pokemon['Sp. Def']*pokemon['HP']
pokemon['HP_adj']=pokemon['SpDef_adj']+pokemon['Defense_adj']

In [13]:
pokemon.head()

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,First_pokemon,Second_pokemon,Total Fights,Win Percentage,Defense_adj,SpDef_adj,HP_adj
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,37.0,37.0,133.0,0.278195,2205,2925,5130
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,46.0,46.0,121.0,0.380165,3780,4800,8580
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,89.0,89.0,132.0,0.674242,6640,8000,14640
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,70.0,70.0,125.0,0.560000,9840,9600,19440
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,55.0,55.0,112.0,0.491071,1677,1950,3627


In [14]:
pokemoves=pd.read_csv('pokemon/Pokemoves.csv')
pokemoves.head()

,Moves,Pokemon,Type,Category,estPower
0,Tackle,Bulbasaur,Normal,Physical,40.0
1,Growl,Bulbasaur,Normal,Status,0.0
2,Vine Whip,Bulbasaur,Grass,Physical,45.0
3,Vine Whip,Bulbasaur,Grass,Physical,45.0
4,Poison Powder,Bulbasaur,Poison,Status,0.0


In [15]:
df = pokemoves.groupby(['Pokemon','Category','Type']).describe()

In [18]:
typelist = pokemon['Type 1'].unique()
catelist = pokemoves['Category'].unique()
for i in typelist:
    for j in catelist:
        if j != 'Status':
            pokemon[str(i)+'_'+str(j)+'_Power'] = 0
        else:
            pokemon[str(i)+'_'+str(j)+'_#'] = 0
for i in pokemon['Name']:
    if ('Mega' not in i):
        for j in typelist:
            for k in catelist:
                if k != 'Status':
                    try:
                        pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_Power'] = df.loc[(i, k, j)][('estPower','max')]
                        if (pokemon[pokemon['Name']==i]['Type 1']==j) or (pokemon[pokemon['Name']==i]['Type 2']==j):
                            pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_Power'] = pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_Power']*1.5
                        else:
                            pass
                    except:
                        pass
                else:
                    try:
                        pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_#'] = df.loc[(i, k, j)][('estPower','count')]            
                    except:
                        pass
    else:
        for j in typelist:
            for k in catelist:
                if k != 'Status':
                    try:
                        pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_Power'] = pokemon.loc[pokemon['Name']==i.split()[1], str(j)+'_'+str(k)+'_Power'].iat[0]
                        if (pokemon[pokemon['Name']==i]['Type 1']==j) or (pokemon[pokemon['Name']==i]['Type 2']==j):
                            pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_Power'] = pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_Power']*1.5
                        else:
                            pass
                    except:
                        pass
                else:
                    try:
                        pokemon.loc[pokemon['Name']==i, str(j)+'_'+str(k)+'_#'] = pokemon.loc[pokemon['Name']==i.split()[1], str(j)+'_'+str(k)+'_#'].iat[0]        
                    except:
                        pass
        

In [21]:
pokemon['Phy_Damage_adj']=0
for i in pokemon['Name']:
    try:
        pokemon.loc[pokemon['Name']==i,'Phy_Damage_adj']=max(pokemon.loc[pokemon['Name']==i,'Grass_Physical_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Fire_Physical_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Water_Physical_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Bug_Physical_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Normal_Physical_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Poison_Physical_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Electric_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Ground_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Fairy_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Fighting_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Psychic_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Rock_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Ghost_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Ice_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Dragon_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Dark_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Steel_Physical_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Flying_Physical_Power'].iat[0])*pokemon.loc[pokemon['Name']==i,'Attack'].iat[0]
    except:
        pass

In [22]:
pokemon['Sp_Damage_adj']=0
for i in pokemon['Name']:
    try:
        pokemon.loc[pokemon['Name']==i,'Sp_Damage_adj']=max(pokemon.loc[pokemon['Name']==i,'Grass_Special_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Fire_Special_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Water_Special_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Bug_Special_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Normal_Special_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Poison_Special_Power'].iat[0], 
                                       pokemon.loc[pokemon['Name']==i,'Electric_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Ground_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Fairy_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Fighting_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Psychic_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Rock_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Ghost_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Ice_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Dragon_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Dark_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Steel_Special_Power'].iat[0],
                                       pokemon.loc[pokemon['Name']==i,'Flying_Special_Power'].iat[0])*pokemon.loc[pokemon['Name']==i,'Sp. Atk'].iat[0]
    except:
        pass

In [24]:
main = pokemon[['Number','Name','Type 1', 'Type 2','HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary', 'Defense_adj',
       'SpDef_adj', 'HP_adj','Phy_Damage_adj','Sp_Damage_adj','Win Percentage']]

In [27]:
main.head(n=20)

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Defense_adj,SpDef_adj,HP_adj,Phy_Damage_adj,Sp_Damage_adj,Win Percentage
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,2205,2925,5130,6370.0,13000.00,0.278195
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,3780,4800,8580,8060.0,16000.00,0.380165
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,6640,8000,14640,11070.0,20000.00,0.674242
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,9840,9600,19440,13500.0,24400.00,0.560000
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,1677,1950,3627,7800.0,7020.00,0.491071
5,6,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,False,3364,3770,7134,9600.0,9360.00,0.542373
6,7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False,6084,6630,12714,12600.0,21800.00,0.864662
7,8,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False,8658,6630,15288,19500.0,26000.00,0.856115
8,9,Mega Charizard Y,Fire,Flying,78,104,78,159,115,100,1,False,6084,8970,15054,15600.0,31800.00,0.844444
9,10,Squirtle,Water,NaN,44,48,65,50,64,43,1,False,2860,2816,5676,6240.0,7500.00,0.162393


In [31]:
main.sort_values(by = ['Win Percentage'], ascending = False ).head(10)

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Defense_adj,SpDef_adj,HP_adj,Phy_Damage_adj,Sp_Damage_adj,Win Percentage
154,155,Mega Aerodactyl,Rock,Flying,80,135,85,70,95,150,1,False,6800,7600,14400,24300.0,9450.0,0.984496
512,513,Weavile,Dark,Ice,70,120,65,45,85,125,4,False,4550,5950,10500,18000.0,6075.0,0.974790
703,704,Tornadus Therian Forme,Flying,NaN,79,100,80,110,90,121,5,True,6320,7110,13430,0.0,0.0,0.968000
19,20,Mega Beedrill,Bug,Poison,65,150,40,15,80,145,1,False,2600,5200,7800,20250.0,3000.0,0.966387
153,154,Aerodactyl,Rock,Flying,80,105,65,60,75,130,1,False,5200,6000,11200,18900.0,8100.0,0.964539
476,477,Mega Lopunny,Normal,Fighting,65,136,94,54,96,135,4,False,6110,6240,12350,20400.0,10800.0,0.961240
726,727,Greninja,Water,Dark,72,95,67,103,71,122,6,False,4824,5112,9936,12825.0,13905.0,0.960630
716,717,Meloetta Pirouette Forme,Normal,Fighting,100,128,90,77,77,128,5,False,9000,7700,16700,0.0,0.0,0.959350
164,165,Mega Mewtwo Y,Psychic,NaN,106,150,70,194,120,140,1,True,7420,12720,20140,10500.0,23280.0,0.952000
349,350,Mega Sharpedo,Water,Dark,70,140,70,110,65,105,3,False,4900,4550,9450,18900.0,14850.0,0.950000
